In [ ]:
filename = r'data/Pre.ht3'

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import phconvert as phc
phc.__version__

# Author

In [ ]:
author = 'Eitan Lerner'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Sample

In [ ]:
comment = 'A demostrative smFRET-nsALEX measurement.'
sample_name = 'Doubly-labeled ssDNA partially hybridized to a complementary strand.'
dye_names = ['ATTO488', 'ATTO647N']
buffer_name = 'Tris20 mM Ph 7.8'

# Prepare data

## Read the data

In [ ]:
d, meta = phc.loader.nsalex_ht3(filename,
                                donor = 0,
                                acceptor = 1,
                                alex_period_donor = (150, 1500),
                                alex_period_acceptor = (1540, 3050),
                                excitation_wavelengths = (470e-9, 635e-9),
                                detection_wavelengths = (525e-9, 690e-9),
                                time_reversed = False)

In [ ]:
fig, ax = plt.subplots(figsize=(13, 4))
phc.plotter.alternation_hist(d, ax=ax)

In [ ]:
detectors = d['photon_data']['detectors']
print('Detector numbers: ', np.unique(detectors))

In [ ]:
print("Detector    Counts")
print("--------   --------")
for det, count in zip(*np.unique(detectors, return_counts=True)):
    print("%8d   %8d" % (det, count))

## Add author and sample info

In [ ]:
d['comment'] = comment

d['sample'] = dict(
    sample_name=sample_name,
    dye_names=[n.encode() for n in dye_names],
    buffer_name=buffer_name,
    num_dyes = len(dye_names))

d['identity'] = dict(
    author=author,
    author_affiliation=author_affiliation,
    creator=creator,
    creator_affiliation=creator_affiliation)

# Validate the Photon-HDF5 structure

Before writing to disk, we assure the file structure follows the Photon-HDF5 format:

In [ ]:
phc.hdf5.assert_valid_photon_hdf5(d)

# Save to Photon-HDF5

In [ ]:
phc.hdf5.save_photon_hdf5(d, close=False)

In [ ]:
#d['_data_file'].close()

## Save PicoQuant metadata

In [ ]:
h5file = d['_data_file']
h5file.create_group('/', 'user')
pq_group = h5file.create_group('/user', 'picoquant')
for key in meta: 
    if np.size(meta[key]) > 0:
        h5file.create_table(pq_group, key, obj=meta[key])

In [ ]:
#h5file.close()

## Print HDF5 file content

In [ ]:
phc.hdf5.print_children(h5file, '/')

In [ ]:
phc.hdf5.print_children(h5file, '/user/picoquant')

In [ ]:
metadata = phc.hdf5.dict_from_group(h5file.root.user.picoquant)

In [ ]:
h5file.close()

# Load Photon-HDF5

In [ ]:
from pprint import pprint

In [ ]:
filename = d['_data_file'].filename

In [ ]:
h5data = phc.hdf5.load_photon_hdf5(filename)

In [ ]:
phc.hdf5.dict_from_group(h5data.identity)

In [ ]:
phc.hdf5.dict_from_group(h5data.setup)

In [ ]:
pprint(phc.hdf5.dict_from_group(h5data.photon_data))